In [1]:
import numpy as np
import pandas as pd
import pypsa
import highspy
#import google.auth
#import pygsheets
import matplotlib.pyplot as plt
from helper_functions import chp_unit_profile, apply_nuclear_outages


plt.rcParams['figure.figsize'] = [15, 9]
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# worksheet details
sheets_scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
credentials, _ = google.auth.default(scopes=sheets_scope)
gs = pygsheets.authorize(custom_credentials=credentials)
sheet = gs.open_by_url("https://docs.google.com/spreadsheets/d/1bdclDHcLE2ttbqhw2-K4DDrMEvDF96HRIv2bTRUCtRk/edit#gid=1663400127")

In [2]:
## read worksheets
load = sheet.worksheet_by_title('load').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('load').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
load_hydrogen = sheet.worksheet_by_title('load_hydrogen').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('load_hydrogen').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
pv = sheet.worksheet_by_title('pv').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('pv').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
wind = sheet.worksheet_by_title('wind').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('wind').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
wind_offshore = sheet.worksheet_by_title('wind_offshore').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('wind_offshore').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
ror = sheet.worksheet_by_title('ror').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('ror').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
inflow_reservoir = sheet.worksheet_by_title('inflow_reservoir').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('inflow_reservoir').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
inflow_hps_ol = sheet.worksheet_by_title('inflow_hps_ol').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('inflow_hps_ol').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
chp = sheet.worksheet_by_title('chp').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('chp').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()
chp_bio = sheet.worksheet_by_title('chp_bio').get_as_df().set_index(pd.to_datetime(sheet.worksheet_by_title('chp_bio').get_as_df()['t'], format="%d/%m/%Y %H:%M:%S")).drop('t', axis=1).dropna(axis=1, how='all').dropna()

buses = sheet.worksheet_by_title('buses').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
cbf_buses = sheet.worksheet_by_title('cbf_buses').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
buses_hydrogen = sheet.worksheet_by_title('buses_hydrogen').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
# buses_reservoir = sheet.worksheet_by_title('buses_reservoir').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
# buses_spillage = sheet.worksheet_by_title('buses_spillage').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
lines = sheet.worksheet_by_title('lines').get_as_df(index_column=2).dropna(axis=1, how='all').dropna()
links = sheet.worksheet_by_title('links').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
links_electrolysis = sheet.worksheet_by_title('links_electrolysis').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
# links_reservoir_turbine = sheet.worksheet_by_title('links_reservoir_turbine').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
# links_reservoir_spillage = sheet.worksheet_by_title('links_reservoir_spillage').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()

gen_pv = sheet.worksheet_by_title('gen_pv').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_wind = sheet.worksheet_by_title('gen_wind').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_wind_offshore = sheet.worksheet_by_title('gen_wind_offshore').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_gas = sheet.worksheet_by_title('gen_gas').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_oil = sheet.worksheet_by_title('gen_oil').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_coal = sheet.worksheet_by_title('gen_coal').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_nuclear = sheet.worksheet_by_title('gen_nuclear').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_biomass = sheet.worksheet_by_title('gen_biomass').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_biogas = sheet.worksheet_by_title('gen_biogas').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_ror = sheet.worksheet_by_title('gen_ror').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
# gen_reservoir = sheet.worksheet_by_title('gen_reservoir').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_other_res = sheet.worksheet_by_title('gen_other_res').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_dsr = sheet.worksheet_by_title('gen_dsr').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_gas_chp = sheet.worksheet_by_title('gen_gas_chp').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_coal_chp = sheet.worksheet_by_title('gen_coal_chp').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_oil_chp = sheet.worksheet_by_title('gen_oil_chp').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_res_chp = sheet.worksheet_by_title('gen_res_chp').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_bio_chp = sheet.worksheet_by_title('gen_bio_chp').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
gen_cbf = sheet.worksheet_by_title('gen_cbf').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
st_hps_cl = sheet.worksheet_by_title('st_hps_cl').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
st_hps_ol = sheet.worksheet_by_title('st_hps_ol').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
st_battery = sheet.worksheet_by_title('st_battery').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
st_hydrogen = sheet.worksheet_by_title('st_hydrogen').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
st_reservoir = sheet.worksheet_by_title('st_reservoir').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
# st_spillage = sheet.worksheet_by_title('st_spillage').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()
st_other = sheet.worksheet_by_title('st_other').get_as_df(index_column=1).dropna(axis=1, how='all').dropna()

NameError: name 'sheet' is not defined

In [2]:
# input parameters
P_MIN_NUCLEAR = 0.4

In [3]:
# TO BE REMOVED ONCE RUNNING - if not running on vm, download input file here https://docs.google.com/spreadsheets/d/1bdclDHcLE2ttbqhw2-K4DDrMEvDF96HRIv2bTRUCtRk/edit#gid=1347891813
DATA_FILE = 'EU-UK-WB-baseline-2030.xlsx'

In [4]:
# TO BE REMOVED ONCE RUNNING
load = pd.read_excel(DATA_FILE, sheet_name='load', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
load_hydrogen = pd.read_excel(DATA_FILE, sheet_name='load_hydrogen', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
pv = pd.read_excel(DATA_FILE, sheet_name='pv', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
wind = pd.read_excel(DATA_FILE, sheet_name='wind', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
wind_offshore = pd.read_excel(DATA_FILE, sheet_name='wind_offshore', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
ror = pd.read_excel(DATA_FILE, sheet_name='ror', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
inflow_reservoir = pd.read_excel(DATA_FILE, sheet_name='inflow_reservoir', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
inflow_hps_ol = pd.read_excel(DATA_FILE, sheet_name='inflow_hps_ol', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
chp = pd.read_excel(DATA_FILE, sheet_name='chp', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
chp_bio = pd.read_excel(DATA_FILE, sheet_name='chp_bio', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()

buses = pd.read_excel(DATA_FILE, sheet_name='buses', index_col=0)
cbf_buses = pd.read_excel(DATA_FILE, sheet_name='cbf_buses', index_col=0)
buses_hydrogen = pd.read_excel(DATA_FILE, sheet_name='buses_hydrogen', index_col=0)
#buses_reservoir = pd.read_excel(DATA_FILE, sheet_name='buses_reservoir', index_col=0)
#buses_spillage = pd.read_excel(DATA_FILE, sheet_name='buses_spillage', index_col=0)

lines = pd.read_excel(DATA_FILE, sheet_name='lines', index_col='name').dropna(axis=1, how='all').dropna()
links = pd.read_excel(DATA_FILE, sheet_name='links', index_col=0, usecols='A:H').dropna(axis=1, how='all').dropna()
links_electrolysis = pd.read_excel(DATA_FILE, sheet_name='links_electrolysis', index_col=0, usecols='A:I').dropna(axis=1, how='all').dropna()
#links_reservoir_turbine = pd.read_excel(DATA_FILE, sheet_name='links_reservoir_turbine', index_col=0).dropna(axis=1, how='all').dropna()
#links_reservoir_spillage = pd.read_excel(DATA_FILE, sheet_name='links_reservoir_spillage', index_col=0).dropna(axis=1, how='all').dropna()

gen_pv = pd.read_excel(DATA_FILE, sheet_name='gen_pv', index_col=0, usecols='A:E')
gen_wind = pd.read_excel(DATA_FILE, sheet_name='gen_wind', index_col=0, usecols='A:E')
gen_wind_offshore = pd.read_excel(DATA_FILE, sheet_name='gen_wind_offshore', index_col=0, usecols='A:E')
gen_gas = pd.read_excel(DATA_FILE, sheet_name='gen_gas', index_col=0, usecols='A:G')
gen_oil = pd.read_excel(DATA_FILE, sheet_name='gen_oil', index_col=0, usecols='A:F')
gen_coal = pd.read_excel(DATA_FILE, sheet_name='gen_coal', index_col=0, usecols='A:G')
gen_nuclear = pd.read_excel(DATA_FILE, sheet_name='gen_nuclear', index_col=0, usecols='A:I')
gen_biomass = pd.read_excel(DATA_FILE, sheet_name='gen_biomass', index_col=0, usecols='A:G')
gen_biogas = pd.read_excel(DATA_FILE, sheet_name='gen_biogas', index_col=0, usecols='A:G')
gen_ror = pd.read_excel(DATA_FILE, sheet_name='gen_ror', index_col=0, usecols='A:E')
#gen_reservoir = pd.read_excel(DATA_FILE, sheet_name='gen_reservoir', index_col=0, usecols='A:E')
gen_other_res = pd.read_excel(DATA_FILE, sheet_name='gen_other_res', index_col=0, usecols='A:F')
gen_dsr = pd.read_excel(DATA_FILE, sheet_name='gen_dsr', index_col=0, usecols='A:F')
gen_gas_chp = pd.read_excel(DATA_FILE, sheet_name='gen_gas_chp', index_col=0, usecols='A:G')
gen_coal_chp = pd.read_excel(DATA_FILE, sheet_name='gen_coal_chp', index_col=0, usecols='A:G')
gen_oil_chp = pd.read_excel(DATA_FILE, sheet_name='gen_oil_chp', index_col=0, usecols='A:F')
gen_res_chp = pd.read_excel(DATA_FILE, sheet_name='gen_res_chp', index_col=0, usecols='A:F')
gen_bio_chp = pd.read_excel(DATA_FILE, sheet_name='gen_bio_chp', index_col=0, usecols='A:F')
gen_cbf = pd.read_excel(DATA_FILE, sheet_name='gen_cbf', index_col=0)

st_hps_cl = pd.read_excel(DATA_FILE, sheet_name='st_hps_cl', index_col=0, usecols='A:I')
st_hps_ol = pd.read_excel(DATA_FILE, sheet_name='st_hps_ol', index_col=0, usecols='A:I')
st_battery = pd.read_excel(DATA_FILE, sheet_name='st_battery', index_col=0, usecols='A:I')
st_hydrogen = pd.read_excel(DATA_FILE, sheet_name='st_hydrogen', index_col=0)
st_reservoir = pd.read_excel(DATA_FILE, sheet_name='st_reservoir', index_col=0, usecols='A:J')
#st_spillage = pd.read_excel(DATA_FILE, sheet_name='st_spillage', index_col=0, usecols='A:E')
st_other = pd.read_excel(DATA_FILE, sheet_name='st_other', index_col=0, usecols='A:I')

In [5]:
# create chp timeseries based on country temperature profile
gas_chp_timeseries=chp_unit_profile(chp, gen_gas_chp)
coal_chp_timeseries=chp_unit_profile(chp, gen_coal_chp)
oil_chp_timeseries=chp_unit_profile(chp, gen_oil_chp)
res_chp_timeseries=chp_unit_profile(chp, gen_res_chp)

In [6]:
# create nuclear time series to replicate maintenance profile
nuclear_p_max_time_series, nuclear_p_min_time_series = apply_nuclear_outages(chp, gen_nuclear, nuclear_p_min=P_MIN_NUCLEAR, french_nucl_cf=0.85, other_nucl_cf=0.95)

In [7]:
network = pypsa.Network()
network.set_snapshots(load.index)

network.import_components_from_dataframe(buses, 'Bus')
network.import_components_from_dataframe(cbf_buses, 'Bus')
network.import_components_from_dataframe(buses_hydrogen, 'Bus')
# network.import_components_from_dataframe(buses_reservoir, 'Bus')
# network.import_components_from_dataframe(buses_spillage, 'Bus')

network.import_components_from_dataframe(lines, 'Line')
network.lines['s_nom_extendable'] = True  

network.madd("Link", links.index,
             bus0=links['bus0'].tolist(), bus1= links['bus1'].tolist(),
             p_nom=links['p_nom'].tolist(), p_max_pu=links['p_max_pu'].to_list())

network.madd("Generator", gen_cbf.index, suffix='_CBF', carrier='CBF',
             bus=gen_cbf.index.tolist(), p_nom=gen_cbf['p_nom'].to_list(), p_nom_extendable=False, 
             marginal_cost=gen_cbf['marginal_cost'].to_list())

dict_carriers = {
    'Lignite': 0.334, 'Hard coal': 0.354, 'Gas': 0.187, 'Gas CCS': 0, 'Oil': 0.334, 'Hydrogen': 0, 'Biomass': 0.403, 'Biogas': 0.178, 'BECCS': 0,
    'Geothermal': 0.026, 'Wind': 0, 'Wind offshore': 0, 'PV': 0, 'HPS_CL': 0, 'HPS_OL': 0, 'ROR': 0, 'Reservoir' :0, 'Reservoir turbine' :0, 'Reservoir spillage':0, 'Reservoir inflow' :0,'Reservoir storage' :0,'Spillage storage' :0, 'Inflow':0, 'Spillage':0, 'Other RES': 0, 'CBF': 0, 'VOLL': 0, 'Battery': 0, 'Other storage' : 0, 'Nuclear': 0, 'Electrolysis':0, 'Hydrogen storage':0, 'Hydrogen for power':0
}

for carrier in dict_carriers:
    network.add("Carrier", name=carrier, co2_emissions=dict_carriers[carrier])

network.madd("Load", load.columns, bus=load.columns, p_set=load)
network.madd("Load", load_hydrogen.columns, bus=load_hydrogen.columns, p_set=load_hydrogen, carrier='Hydrogen')

Index(['Austria_H2', 'Belgium_H2', 'Bulgaria_H2', 'Croatia_H2', 'Czechia_H2',
       'Denmark_H2', 'Estonia_H2', 'Finland_H2', 'France_H2', 'Germany_H2',
       'Greece_H2', 'Hungary_H2', 'Ireland_H2', 'Italy_H2', 'Latvia_H2',
       'Lithuania_H2', 'Netherlands_H2', 'Poland_H2', 'Portugal_H2',
       'Romania_H2', 'Slovakia_H2', 'Slovenia_H2', 'Spain_H2', 'Sweden_H2',
       'UK_H2', 'Norway_H2', 'Switzerland_H2', 'Albania_H2',
       'BosniaHerzegovina_H2', 'Kosovo_H2', 'Moldova_H2', 'Montenegro_H2',
       'NorthMacedonia_H2', 'Serbia_H2', 'Ukraine_H2'],
      dtype='object')

In [8]:
network.madd('Generator', 
      gen_pv['bus'],
      suffix='_PV',
      bus=gen_pv['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_pv['p_nom'].to_list(),
      carrier='PV', 
      marginal_cost=gen_pv['marginal_cost'].to_list(), 
      p_max_pu=pv)

network.madd('Generator', 
      gen_wind['bus'],
      suffix='_Wind',
      bus=gen_wind['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind['p_nom'].to_list(), 
      carrier='Wind', 
      marginal_cost=gen_wind['marginal_cost'].to_list(), 
      p_max_pu=wind)

network.madd('Generator', 
      gen_wind_offshore['bus'],
      suffix='_Wind_offshore',
      bus=gen_wind_offshore['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind_offshore['p_nom'].to_list(), 
      carrier='Wind offshore', 
      marginal_cost=gen_wind_offshore['marginal_cost'].to_list(), 
      p_max_pu=wind_offshore)


network.madd('Generator', 
      gen_gas.index,
      bus=gen_gas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_gas['p_nom'].to_list(), 
      carrier=gen_gas['carrier'].to_list(), 
      marginal_cost=gen_gas['marginal_cost'].to_list(),
      efficiency=gen_gas['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_oil.index,
      bus=gen_oil['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil['p_nom'].to_list(), 
      carrier='Oil', 
      marginal_cost=gen_oil['marginal_cost'].to_list(),
      efficiency=gen_oil['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal.index,
      bus=gen_coal['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal['p_nom'].to_list(), 
      carrier=gen_coal['carrier'].to_list(), 
      marginal_cost=gen_coal['marginal_cost'].to_list(),
      efficiency=gen_coal['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_nuclear.index,
      bus=gen_nuclear['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_nuclear['p_nom'].to_list(), 
      carrier='Nuclear', 
      marginal_cost=gen_nuclear['marginal_cost'].to_list(),
      efficiency=gen_nuclear['efficiency'].to_list(),
      p_max_pu=nuclear_p_max_time_series,
      p_min_pu=nuclear_p_min_time_series
      )

network.madd('Generator', 
      gen_biomass.index,
      bus=gen_biomass['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biomass['p_nom'].to_list(), 
      carrier=gen_biomass['carrier'].to_list(), 
      marginal_cost=gen_biomass['marginal_cost'].to_list(),
      efficiency=gen_biomass['efficiency'].to_list(),
      p_max_pu=gen_biomass['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_biogas['bus'],
      suffix='_Biogas',
      bus=gen_biogas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biogas['p_nom'].to_list(), 
      carrier='Biogas', 
      marginal_cost=gen_biogas['marginal_cost'].to_list(),
      efficiency=gen_biogas['efficiency'].to_list(),
      p_max_pu=gen_biogas['p_max_pu'].to_list()
      )

network.madd('Generator', 
       gen_ror['bus'],
       suffix='_ROR',
       bus=gen_ror['bus'].to_list(), 
       p_nom_extendable=False, 
       p_nom=gen_ror['p_nom'].to_list(), 
       carrier=gen_ror['carrier'].to_list(),
       marginal_cost=gen_ror['marginal_cost'].to_list(),
       p_max_pu=ror
       )

# network.madd('Generator', 
#        gen_reservoir['bus'],
#        suffix='_inflow',
#        bus=gen_reservoir['bus'].to_list(), 
#        p_nom_extendable=False, 
#        p_nom=gen_reservoir['p_nom'].to_list(), 
#        carrier=gen_reservoir['carrier'].to_list(),
#        p_max_pu=inflow_reservoir
#        )

network.madd('Generator', 
      gen_other_res['bus'],
      suffix='_OtherRES',
      bus=gen_other_res['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_other_res['p_nom'].to_list(), 
      carrier='Other RES', 
      marginal_cost=gen_other_res['marginal_cost'].to_list(),
      p_max_pu=gen_other_res['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_dsr['bus'],
      suffix='_DSR',
      bus=gen_dsr['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_dsr['p_nom'].to_list(), 
      carrier='DSR', 
      marginal_cost=gen_dsr['marginal_cost'].to_list(),
      p_max_pu=gen_dsr['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_gas_chp.index,
      bus=gen_gas_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_gas_chp['p_nom'].to_list(), 
      carrier=gen_gas_chp['carrier'].to_list(), 
      marginal_cost=gen_gas_chp['marginal_cost'].to_list(),
      p_max_pu=gas_chp_timeseries,
      p_min_pu=0.9*gas_chp_timeseries,
      efficiency=gen_gas_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal_chp.index,
      bus=gen_coal_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal_chp['p_nom'].to_list(), 
      carrier=gen_coal_chp['carrier'].to_list(), 
      marginal_cost=gen_coal_chp['marginal_cost'].to_list(),
      p_max_pu=coal_chp_timeseries,
      p_min_pu=0.9*coal_chp_timeseries,
      efficiency=gen_coal_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_oil_chp.index,
      bus=gen_oil_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil_chp['p_nom'].to_list(), 
      carrier=gen_oil_chp['carrier'].to_list(), 
      marginal_cost=gen_oil_chp['marginal_cost'].to_list(),
      p_max_pu=oil_chp_timeseries,
      p_min_pu=0.9*oil_chp_timeseries,
      efficiency=gen_oil_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_res_chp.index,
      bus=gen_res_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_res_chp['p_nom'].to_list(), 
      carrier=gen_res_chp['carrier'].to_list(), 
      marginal_cost=gen_res_chp['marginal_cost'].to_list(),
      p_max_pu=res_chp_timeseries,
      p_min_pu=0.9*res_chp_timeseries,
      efficiency=gen_res_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_bio_chp.index,
      bus=gen_bio_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_bio_chp['p_nom'].to_list(), 
      carrier=gen_bio_chp['carrier'].to_list(), 
      marginal_cost=gen_bio_chp['marginal_cost'].to_list(),
      p_max_pu=chp_bio,
      p_min_pu=chp_bio*0.9,
      efficiency=gen_bio_chp['efficiency'].to_list()
      )

network.madd("StorageUnit", st_hps_cl.index, bus=st_hps_cl['bus'].tolist(), carrier=st_hps_cl['carrier'].tolist(),
             p_nom=st_hps_cl['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_hps_cl['max_hours'].to_list(),
             p_max_pu=st_hps_cl['p_max_pu'].tolist(),
             efficiency_dispatch=st_hps_cl['efficiency_dispatch'].tolist(),
             standing_loss=st_hps_cl['standing_loss'].tolist())

network.madd("StorageUnit", st_hps_ol['bus'].tolist(), suffix='_HPS_OL', bus=st_hps_ol['bus'].tolist(), carrier=st_hps_ol['carrier'].tolist(),
             p_nom=st_hps_ol['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_hps_ol['max_hours'].to_list(),
             p_max_pu=inflow_hps_ol,
             efficiency_dispatch=st_hps_ol['efficiency_dispatch'].tolist(),
             standing_loss=st_hps_ol['standing_loss'].tolist(), state_of_charge_initial=st_hps_ol['state_of_charge_initial'].tolist())
            
network.madd("StorageUnit", st_reservoir['bus'].tolist(), suffix='_Reservoir', bus=st_reservoir['bus'].tolist(), carrier=st_reservoir['carrier'].tolist(),
             p_nom=st_reservoir['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_reservoir['max_hours'].to_list(),
             p_max_pu=inflow_reservoir,
             efficiency_dispatch=st_reservoir['efficiency_dispatch'].tolist(), efficiency_store=st_reservoir['efficiency_store'].tolist(),
             standing_loss=st_reservoir['standing_loss'].tolist(), state_of_charge_initial=st_reservoir['state_of_charge_initial'].tolist())

network.madd("StorageUnit", st_battery.index, bus=st_battery['bus'].tolist(), carrier=st_battery['carrier'].tolist(),
             p_nom=st_battery['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_battery['max_hours'].to_list(),
             p_max_pu=st_battery['p_max_pu'].tolist(),
             efficiency_dispatch=st_battery['efficiency_dispatch'].tolist(),
             standing_loss=st_battery['standing_loss'].tolist())

network.madd("StorageUnit", st_other.index, bus=st_other['bus'].tolist(), carrier=st_other['carrier'].tolist(),
             p_nom=st_other['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_other['max_hours'].to_list(),
             p_max_pu=st_other['p_max_pu'].tolist(),
             efficiency_dispatch=st_other['efficiency_dispatch'].tolist(),
             standing_loss=st_other['standing_loss'].tolist())

network.madd("Link", links_electrolysis.index,
             bus0=links_electrolysis['bus0'].tolist(), bus1= links_electrolysis['bus1'].tolist(),
             p_nom=links_electrolysis['p_nom'].tolist(), p_nom_extendable=links_electrolysis['p_nom_extendable'].tolist(), carrier=links_electrolysis['carrier'].tolist(), 
             efficiency=links_electrolysis['efficiency'].tolist())

network.madd("Store", st_hydrogen.index, bus=st_hydrogen['bus'].tolist(), carrier=st_hydrogen['carrier'].tolist(),
             e_nom=st_hydrogen['e_nom'].tolist(), e_nom_extendable=False, e_cyclic=True)

# network.madd("Link", links_reservoir_turbine.index,
#              bus0=links_reservoir_turbine['bus0'].tolist(), bus1= links_reservoir_turbine['bus1'].tolist(),
#              p_nom=links_reservoir_turbine['p_nom'].tolist(), p_nom_extendable=links_reservoir_turbine['p_nom_extendable'].tolist(), carrier=links_reservoir_turbine['carrier'].tolist(), 
#              efficiency=links_reservoir_turbine['efficiency'].tolist(), marginal_cost=links_reservoir_turbine['marginal_cost'].tolist())

# network.madd("Link", links_reservoir_spillage.index,
#              bus0=links_reservoir_spillage['bus0'].tolist(), bus1= links_reservoir_spillage['bus1'].tolist(),
#              p_nom=links_reservoir_spillage['p_nom'].tolist(), p_nom_extendable=links_reservoir_spillage['p_nom_extendable'].tolist(), carrier=links_reservoir_spillage['carrier'].tolist(), 
#              efficiency=links_reservoir_spillage['efficiency'].tolist(), marginal_cost=links_reservoir_spillage['marginal_cost'].tolist())

# network.madd("Store", st_reservoir.index, bus=st_reservoir['bus'].tolist(), carrier=st_reservoir['carrier'].tolist(),
#              e_nom=st_reservoir['e_nom'].tolist(), e_nom_extendable=False, e_initial=st_reservoir['e_initial'].tolist())

# network.madd("Store", st_spillage.index, bus=st_spillage['bus'].tolist(), carrier=st_spillage['carrier'].tolist(),
#              e_nom=st_spillage['e_nom'].tolist(), e_nom_extendable=False, e_initial=st_spillage['e_initial'].tolist())

Index(['Austria_H2_Storage', 'Belgium_H2_Storage', 'Bulgaria_H2_Storage',
       'Croatia_H2_Storage', 'Czechia_H2_Storage', 'Denmark_H2_Storage',
       'Estonia_H2_Storage', 'Finland_H2_Storage', 'France_H2_Storage',
       'Germany_H2_Storage', 'Greece_H2_Storage', 'Hungary_H2_Storage',
       'Ireland_H2_Storage', 'Italy_H2_Storage', 'Latvia_H2_Storage',
       'Lithuania_H2_Storage', 'Netherlands_H2_Storage', 'Poland_H2_Storage',
       'Portugal_H2_Storage', 'Romania_H2_Storage', 'Slovakia_H2_Storage',
       'Slovenia_H2_Storage', 'Spain_H2_Storage', 'Sweden_H2_Storage',
       'UK_H2_Storage', 'Norway_H2_Storage', 'Switzerland_H2_Storage',
       'Albania_H2_Storage', 'BosniaHerzegovina_H2_Storage',
       'Kosovo_H2_Storage', 'Moldova_H2_Storage', 'Montenegro_H2_Storage',
       'NorthMacedonia_H2_Storage', 'Serbia_H2_Storage', 'Ukraine_H2_Storage'],
      dtype='object', name='name')

In [9]:
network.consistency_check()

In [10]:
network.iplot()

{'data': [{'x': Bus
   Austria              14.371022
   Belgium               4.524273
   Bulgaria             25.320274
   Croatia              14.907862
   Czechia              18.429565
                          ...    
   Moldova_H2           28.369900
   Montenegro_H2        19.374400
   NorthMacedonia_H2    21.745300
   Serbia_H2            21.005900
   Ukraine_H2           31.165600
   Name: x, Length: 72, dtype: float64,
   'y': Bus
   Austria              47.631732
   Belgium              50.378963
   Bulgaria             42.533355
   Croatia              45.069383
   Czechia              49.864546
                          ...    
   Moldova_H2           47.411600
   Montenegro_H2        42.708700
   NorthMacedonia_H2    41.608600
   Serbia_H2            44.016500
   Ukraine_H2           48.379400
   Name: y, Length: 72, dtype: float64,
   'text': Index(['Bus Austria', 'Bus Belgium', 'Bus Bulgaria', 'Bus Croatia',
          'Bus Czechia', 'Bus Denmark', 'Bus Estonia', 'Bus F

In [11]:
network.optimize(solver_name='highs')

<__array_function__ internals>:200: RuntimeWarning:

invalid value encountered in cast

<__array_function__ internals>:200: RuntimeWarning:

invalid value encountered in cast

<__array_function__ internals>:200: RuntimeWarning:

invalid value encountered in cast

<__array_function__ internals>:200: RuntimeWarning:

invalid value encountered in cast

INFO:linopy.model: Solve linear problem using Highs solver
INFO:linopy.io:Writing objective.
Writing variables.: 100%|██████████| 8/8 [00:40<00:00,  5.01s/it]
INFO:linopy.io: Writing time: 274.45s
INFO:linopy.solvers:Log file at C:\Users\Pawel\AppData\Local\Temp\highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 24370320 primals, 50597760 duals
Objective: 1.26e+11
Solver model: available
Solver message: optimal



('ok', 'optimal')

In [12]:
# generation by fuel
p_by_carrier = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum()
generation_by_fuel_yearly = p_by_carrier.sum()/1000000
generation_by_fuel_yearly

carrier
BECCS              3.497347
Biogas           111.355267
Biomass          180.819389
CBF                0.000000
DSR               48.999875
Gas              496.635580
Hard coal         60.391812
Lignite           55.686475
Nuclear          754.693842
Oil               44.434050
Other RES         73.074597
PV               788.321103
ROR              211.460018
Wind             905.624982
Wind offshore    543.115236
dtype: float64

In [15]:
# Generation by fuel by country with exports to csv
p_by_carrier = network.generators_t.p.groupby([network.generators.carrier, network.generators.bus], axis=1).sum()
generation_by_fuel_by_country = p_by_carrier.sum()/1000000
generation_by_fuel_by_country.to_csv('generation_by_fuel_by_country.csv')

In [16]:
# Generation by unit with exports to csv
p_by_unit = network.generators_t.p.sum()/1000000
generation_by_unit = p_by_unit
generation_by_unit.to_csv('generation_by_unit.csv', encoding='utf-8-sig')

In [17]:
# Interconnector flows with exports to csv
generation_by_link = network.links_t.p0.sum()/1000000
generation_by_link.to_csv('generation_by_link.csv')

In [18]:
# Capacity by fuel
capacity_by_fuel = network.generators.groupby("carrier")["p_nom"].sum()
capacity_by_fuel

carrier
BECCS               550.000000
Biogas            14252.919104
Biomass           23887.795958
CBF              100000.000000
DSR              150284.340308
Gas              227238.070000
Hard coal         53161.000000
Hydro            186210.000000
Lignite           37355.000000
Nuclear          111074.000000
Oil               29323.000000
Other RES         19578.378009
PV               686651.000000
Wind             415251.000000
Wind offshore    152030.000000
Name: p_nom, dtype: float64

In [19]:
# Capacity by fuel by country with exports to csv
capacity_by_fuel = network.generators.groupby(["carrier","bus"])["p_nom"].sum()
capacity_by_fuel.to_csv('capacity_by_fuel_by_country.csv')

In [20]:
# prices with export to csv
prices = network.buses_t.marginal_price
prices.to_csv('prices.csv')

In [88]:
# curtailment calcs
pv_potential = pv.copy()
wind_potential = wind.copy()
wind_offshore_potential = wind_offshore.copy()
for i,v in enumerate(pv.columns):
    pv_potential.loc[:,v] = pv_potential.loc[:,v]*gen_pv.iloc[i,2]
    wind_potential.loc[:,v] = wind_potential.loc[:,v]*gen_wind.iloc[i,2]
    wind_offshore_potential.loc[:,v] = wind_offshore_potential.loc[:,v]*gen_wind_offshore.iloc[i,2]

solar_curtailment = pv_potential - p_by_carrier.iloc[:, p_by_carrier.columns.get_level_values(0)=='PV'].droplevel(level=0, axis=1)
onshore_curtailment = wind_potential - p_by_carrier.iloc[:, p_by_carrier.columns.get_level_values(0)=='Wind'].droplevel(level=0, axis=1)
offshore_curtailment = wind_offshore_potential - p_by_carrier.iloc[:, p_by_carrier.columns.get_level_values(0)=='Wind offshore'].droplevel(level=0, axis=1)

annual_curtailment = pd.concat([solar_curtailment.sum().to_frame(name='PV_curtailment')/1000000,onshore_curtailment.sum().to_frame(name='Wind_curtailment')/1000000,
          offshore_curtailment.sum().to_frame(name='Wind_offshore_curtailment')/1000000],axis=1)
annual_curtailment['PV_potential'] = pv_potential.sum()/1000000
annual_curtailment['Wind_potential'] = wind_potential.sum()/1000000
annual_curtailment['Wind_offshore_potential'] = wind_offshore_potential.sum()/1000000
annual_curtailment['Solar_pct'] = annual_curtailment['PV_curtailment']/annual_curtailment['PV_potential']
annual_curtailment['Wind_pct'] = annual_curtailment['Wind_curtailment']/annual_curtailment['Wind_potential']
annual_curtailment['Wind_offshore_pct'] = annual_curtailment['Wind_offshore_curtailment']/annual_curtailment['Wind_offshore_potential']

In [90]:
annual_curtailment.to_csv("curtailment_RES.csv")

In [13]:
# storage 
state_of_charge = network.storage_units_t.state_of_charge.to_csv("state_of_charge.csv")
storage_power = network.storage_units_t.p.to_csv("storage_power_output.csv")
storage_power_dispatch = network.storage_units_t.p_dispatch.to_csv("storage_dispatched.csv")

In [14]:
# hourly dsf
gen_by_fuel = network.generators_t.p.filter(like='_DSR').to_csv("hourly_dsf.csv")

In [58]:
# hourly eu generation by fuel
generation_t = network.generators_t.p.groupby([network.generators.carrier, network.generators.bus], axis=1).sum()
eu_hourly_generation = generation_t.loc[:,~generation_t.columns.get_level_values(1).isin(['Albania', 'BosniaHerzegovina', 'Kosovo', 'Moldova', 'Montenegro', 'NorthMacedonia', 'Norway', 'Serbia', 'Switzerland' ,'UK'])]
eu_hourly_by_fuel = eu_hourly_generation.groupby('carrier',axis=1).sum()
eu_hourly_by_fuel.to_csv("eu_hourly_by_fuel.csv")

In [59]:
# save input worksheet to repo
with pd.ExcelWriter('input_data.xlsx', engine='openpyxl') as writer:
    for wks in sheet.worksheets():
        df = wks.get_as_df()
        df.to_excel(writer, sheet_name=wks.title, index=False)

/home/harri/venv/lib/python3.9/site-packages/pygsheets/worksheet.py:1554: UserWarning:

At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.

/home/harri/venv/lib/python3.9/site-packages/pygsheets/worksheet.py:1554: UserWarning:

At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.



In [60]:
network.export_to_netcdf(path="network.nc")

INFO:pypsa.io:Exported network network.nc has carriers, storage_units, loads, generators, stores, links, buses


<xarray.Dataset>
Dimensions:                            (snapshots: 8760, investment_periods: 0,
                                        carriers_i: 24, storage_units_i: 105,
                                        storage_units_t_p_i: 61,
                                        storage_units_t_p_dispatch_i: 61,
                                        storage_units_t_p_store_i: 61,
                                        storage_units_t_state_of_charge_i: 105,
                                        ...
                                        generators_t_p_i: 1317, stores_i: 35,
                                        stores_t_p_i: 1, stores_t_e_i: 1,
                                        links_i: 201, links_t_p0_i: 162,
                                        links_t_p1_i: 162, buses_i: 72,
                                        buses_t_p_i: 36,
                                        buses_t_marginal_price_i: 38)
Coordinates: (12/24)
  * snapshots                          (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                 (investment_periods) int64 
  * carriers_i                         (carriers_i) object 'Lignite' ... 'Hyd...
  * storage_units_i                    (storage_units_i) object 'Austria_HPS'...
  * storage_units_t_p_i                (storage_units_t_p_i) object 'Austria_...
  * storage_units_t_p_dispatch_i       (storage_units_t_p_dispatch_i) object ...
    ...                                 ...
  * links_i                            (links_i) object 'link_AL_XK' ... 'Ukr...
  * links_t_p0_i                       (links_t_p0_i) object 'link_AL_XK' ......
  * links_t_p1_i                       (links_t_p1_i) object 'link_AL_XK' ......
  * buses_i                            (buses_i) object 'Austria' ... 'Ukrain...
  * buses_t_p_i                        (buses_t_p_i) object 'Austria' ... 'UK...
  * buses_t_marginal_price_i           (buses_t_marginal_price_i) object 'Rus...
Data variables: (12/54)
    snapshots_snapshot                 (snapshots) datetime64[ns] 2030-01-01 ...
    snapshots_objective                (snapshots) float64 1.0 1.0 ... 1.0 1.0
    snapshots_stores                   (snapshots) float64 1.0 1.0 ... 1.0 1.0
    snapshots_generators               (snapshots) float64 1.0 1.0 ... 1.0 1.0
    investment_periods_objective       (investment_periods) object 
    investment_periods_years           (investment_periods) object 
    ...                                 ...
    buses_v_nom                        (buses_i) float64 380.0 380.0 ... 1.0 1.0
    buses_x                            (buses_i) float64 14.37 4.524 ... 31.17
    buses_y                            (buses_i) float64 47.63 50.38 ... 48.38
    buses_carrier                      (buses_i) object 'AC' 'AC' ... 'Hydrogen'
    buses_t_p                          (snapshots, buses_t_p_i) float64 -9.09...
    buses_t_marginal_price             (snapshots, buses_t_marginal_price_i) float64 ...
Attributes:
    network__linearized_uc:  0
    network__multi_invest:   0
    network_name:            
    network_objective:       56864065902.05347
    network_pypsa_version:   0.25.2
    network_srid:            4326
    meta:                    {}